# Test focusing on just LIMS Orders (different formats for orders)
# and specifically those orders that use AAL LIMS

```
Focusing on LIMS Order formats.
Specifically there are a couple ways to place (and cancel) orders.
Many of the fields are optional (including AAL sub-fields).

Rules apply with respect to what happens when a field is (or is not) present.

Test cases expect to hit:

Command, LIMS Id, Serial#, SampleID, Lot, Method, Handling Rule, Action,

  ORDER,  LimsID,        , SampleID,    ,       ,              ,       ,
  ORDER,  LimsID,        , SampleId,

```

# IT DOES NOT COVER:
```


```


In [235]:
class AAL:
    
    def __init__(self):
        self.things_to_test = []
        self.is_valid = {}
        self.is_enabled = {}
        self.value = {}

    
    def add(self, name, is_valid, is_enabled, value):
        self.things_to_test.append(name)
        self.is_valid[name] = is_valid
        self.is_enabled[name] = is_enabled
        self.value[name] = value
      

    

In [236]:
import io
import os
from datetime import datetime


dirname = "./dir3_request_for_aal/"
if not os.path.exists(dirname):
    os.mkdir(dirname)
filename = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

aal = AAL()
aal.add(name="foo", is_valid=False, is_enabled=False, value=None)
aal.add(name="-2",  is_valid=False, is_enabled=False, value=None)
aal.add(name="-1",  is_valid=True,  is_enabled=False, value=None)
aal.add(name=" ",   is_valid=True,  is_enabled=False, value=None)
#aal.add(name="",    is_valid=True,  is_enabled=False, value=None)
#aal.add(name="0",   is_valid=True,  is_enabled=True,  value=0)
aal.add(name="1",   is_valid=True,  is_enabled=True,  value=1)
aal.add(name="2",   is_valid=True,  is_enabled=True,  value=2)
#aal.add(name="3",   is_valid=True,  is_enabled=True,  value=3)
#aal.add(name="4",   is_valid=True,  is_enabled=True,  value=4)
#aal.add(name= "99999999",   is_valid=True,  is_enabled=True,  value=99999999)
#aal.add(name="100000000",   is_valid=False,  is_enabled=False,  value=None)
            

filenamegood        = dirname + filename + "_good.request.txt"
filenamebad         = dirname + filename + "_bad.request.txt"
filenamecancel      = dirname + filename + "_cancel.request.txt"
filenameuniqueaals  = dirname + filename + "_unique_aals.txt"
filenamereadme      = dirname + filename + "_readme.txt"

file_good = open(filenamegood, 'w')
file_bad = open(filenamebad, 'w')
file_cancel = open(filenamecancel, 'w')
file_unique_aals = open(filenameuniqueaals, 'w')
file_readme = open(filenamereadme, 'w')

i = -1
count_good = 0
count_bad = 0
list_of_aals = []

for alert in aal.things_to_test:
    for action in aal.things_to_test:
        for spec in aal.things_to_test:
            for passed in aal.things_to_test:
                i = i + 1

                is_bad = False

                # model - the rules
                
                # if any of the fields are not valid (-2 or "foo")
                if not aal.is_valid[alert] or not aal.is_valid[action] or not aal.is_valid[spec] or not aal.is_valid[passed]:
                    is_bad = is_bad | True 

                # spec and alert exclude one-another
                if aal.is_enabled[alert] and aal.is_enabled[spec]:
                    is_bad = is_bad | True

                # spec and action exclude one-another
                if aal.is_enabled[action] and aal.is_enabled[spec]:
                    is_bad = is_bad |True

                # if alert and action are both enabled, alert needs to be < action
                if (aal.is_enabled[alert] and aal.is_enabled[action]) and (aal.value[alert] >= aal.value[action]):
                    is_bad = is_bad |True

                    
                # model - track the good/bad count, and also the number of AALs
                if is_bad:
                    count_bad = count_bad + 1
                    comment = "BAD  " + comment                        
                else:
                    count_good = count_good + 1
                    this_aal = [aal.value[alert], aal.value[action], aal.value[spec], aal.value[passed]]
                    if not this_aal in list_of_aals:
                        list_of_aals.append([aal.value[alert], aal.value[action], aal.value[spec], aal.value[passed]])
                    comment = "GOOD " + comment
                    

                # create an order
                command   = "ORDER"
                lims_id   = "LID" + str(i).zfill(3)
                serial_n  = ""
                sample_id = ""
                lot_num   = ""
                meth_num  = "M1"
                handle_r  = "HR1"
                action_a  = alert + ";" + action + ";" + spec + ";" + passed
                order_command  = command + "," + lims_id + "," + serial_n + "," + sample_id + "," + lot_num + "," + meth_num + "," + handle_r + "," + action_a + ","

                command   = "CANCEL"
                lims_id   = "LID" + str(i).zfill(3)
                serial_n  = ""
                sample_id = ""
                lot_num   = ""
                meth_num  = ""
                handle_r  = ""
                action_a  = ""
                cancel_command  = command + "," + lims_id + "," + serial_n + "," + sample_id + "," + lot_num + "," + meth_num + "," + handle_r + "," + action_a + ","

                if is_bad:
                    file_bad.write(order_command + "ORIGINAL(BAD)-->" + order_command + "\n")
                else:
                    file_good.write(order_command + "ORIGINAL(GOOD)-->" + order_command + "\n")
                    file_cancel.write(cancel_command + "ORIGINAL-->" + cancel_command + "\n")

for a in list_of_aals:
    file_unique_aals.write(str(a) + '\n')


# make a README
file_readme.write('This test iterates through the following values for Alert;Action;Spec;Pass fields in the LIMS AAL order:\n\n')
for name in aal.things_to_test:
    file_readme.write('"' + name + '"\n')

file_readme.write("\nExpected results:\n")
file_readme.write("number of orders accepted = " + str(count_good) + '\n')
file_readme.write("number of orders rejected = " + str(count_bad) + '\n')
file_readme.write("number of unique AALs = " + str(len(list_of_aals)) + '\n')


file_good.close()
file_bad.close()
file_cancel.close()
file_unique_aals.close()
file_readme.close()

print("Expected results:")
print("number of orders accepted = " + str(count_good))
print("number of orders rejected = " + str(count_bad))
print("number of unique AALs = " + str(len(list_of_aals)))


Expected results:
number of orders accepted = 136
number of orders rejected = 1160
number of unique AALs = 24
